# Wykład Algorytmy w Inżynierii Danych 2021L
## Temat: Wartości własne - część 1

Opracownie: Robert Szmurło

Przykładowa deklaracja macierzy $2 \times 2$

In [1]:
A = [1 2;4 5]

2×2 Array{Int64,2}:
 1  2
 4  5

Chcemy używać funkcji z bibliotek dotyczących algebry liniowej (`eigen`) oraz statystyki (`cov`)

In [2]:
using LinearAlgebra
using Statistics

Sprawdzamy jak działa wbudowana funkcja do wyznaczania wartości własnych.

In [3]:
e,v = eigen(A);

Sprawdzamy jak znaleźć maksymalną wartość co do modułu: 

$$max( |e_i| ) \text{  dla  } i=1,2,3,\ldots,n$$ 

In [4]:
maximum(abs.(e))

6.464101615137754

Liczymy przykładową wartość wektorów kierunkowych PCA i rzutujemy na nie wektor danych wejściowych. Zazwyczaj wybieramy tylko te wektory włąsne, które są związane z najwiękzymi co do modułu wartościami własnymi.

$$\Sigma ={\begin{bmatrix}\sigma _{1}^{2}&\sigma _{12}&\cdots &\sigma _{1n}\\\sigma _{21}&\sigma _{2}^{2}&\cdots &\sigma _{2n}\\\vdots &\cdots &\ddots &\vdots \\\sigma _{n1}&\sigma _{n2}&\cdots &\sigma _{n}^{2}\end{bmatrix}}$$

gdzie:

$\sigma _{i}^{2}=D^{2}X_{i}$ - wariancja zmiennej $X_i$,  
$\sigma _{ij}=\operatorname {cov} (X_{i},X_{j})$ - kowariancja między zmiennymi $X_i$ i $X_j$

Obliczone wektory własne reprezentują nam przestrzeń rzutowania PCA. Liczymy zatem: 

$$P=\text{vectors}^T \cdot X^T$$

In [5]:
X = [-2 -2; 0 0; 2 2];
C = cov(X)
values, vectors = eigen(C)
println("Wartości własne: ",values)
println("Wektory własne: ", vectors)
P = vectors' * X' # tutaj rzutujemy
P'

Wartości własne: [0.0, 8.0]
Wektory własne: [-0.7071067811865475 0.7071067811865475; 0.7071067811865475 0.7071067811865475]


3×2 Adjoint{Float64,Array{Float64,2}}:
 0.0  -2.82843
 0.0   0.0
 0.0   2.82843

Jak obliczać wartości własne za pomocą wielomianu charakterystycznego używająć wbudowanych funkcji do znajdowania pierwiastków. Samodzielnie tylko napiszemy funkcję obliczającą współczynniki wielomianu charakterystycznego korzystając z metody Faddeeva-Laverriera:

$$(-1)^n [\lambda^n - p_1 \lambda^{n-1} - p_2 \lambda^{n-2} - \ldots - p_{n-1} \lambda - p_n] = 0$$

$$\begin{align*}
[P_1] &= [A], p_1=trace [P_1]; \\
[P_2] &= [A]([P_1] - p_1[I]), p_2=\frac{1}{2}trace[P_2]; \\
[P_3] &= [A]([P_2] - p_2[I]), p_3=\frac{1}{3}trace[P_3]; \\
&\vdots \\
[P_i] &= [A]([P_{i-1}] - p_{i-1}[I]), p_i=\frac{1}{i}trace[P_i]; \\
&\vdots \\
[P_n] &= [A]([P_{n-1}] - p_{n-1}[I]), p_n=\frac{1}{n}trace[P_n];
\end{align*}$$


In [6]:
using Polynomials

In [7]:
A = [12 6 -6
    6 16 2
    -6 2 16];

In [8]:
n = size(A,1) # tylko sprawdzamy czy w Julia size działa tak samo jak w MATLABie :-)

3

In [9]:
n = size(A,1);

# metoda Fadeeva-Laveriera
p = [(-1)^n];
i = 1;
P = A;
for i =1:n
    push!(p, 1/i*tr(P) );
    P = A*(P - p[i+1]*I);
end

# odwracamy współczynniki, ponieważ Polynomials w Julia 
# pierwszy element wektora traktują jako wartośc przy 
# wykładniku potęgi równym 0
coefficients = p[end:-1:1]
print(coefficients)

poly=Polynomial(coefficients)
print(poly)

# liczymy pierwiastki (zera) wielomianu charakterytycznego - czyli wartości własne
root = roots(poly)
print(root)

# sprawdźmy czy eigen zwróci to samo
eigen(A)

[1728, -564, 44, -1]1728 - 564*x + 44*x^2 - x^3[4.455996254682469, 18.000000000000064, 21.54400374531746]

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
3-element Array{Float64,1}:
  4.455996254682473
 18.000000000000004
 21.544003745317532
vectors:
3×3 Array{Float64,2}:
  0.747342  3.33067e-16   0.664439
 -0.469829  0.707107      0.528451
  0.469829  0.707107     -0.528451

Trochę brakuje w Julia funkcji `eye(n)` z MATLABA. Niby Można używać `I`, ale on jest niestety niemutowalny i tworzenie macierzy permutacji (lub obrotu jak w metodzie Jakobiego, czy też Givensa) nie jest zbyt wygodne. 

Zróbmy ją sobie zatem sami.

In [10]:
function eye(n)
    return Matrix{Float64}(I,n,n)
end

eye (generic function with 1 method)

Funkcja zwracająca indeksy (wspólrzędne indeksowe) maksymalnych co do modułu elementów pozadiagonalnych. 

In [11]:
function maxst(A)
    s = 1;
    t = 2;
    n = size(A,1);
    
    for c = 2:n
        for r = 1:c-1
            if abs(A[r,c]) > abs(A[s,t])
                s = r;
                t = c;
            end
        end
    end
    return s,t
end

maxst (generic function with 1 method)

Upewniamy sie jak działa w Julia funkcja `signum`.

In [12]:
sign(-5)

-1

Naiwna implementacja metody diagonalizacji macierzy za pomocą obrotów Jakobiego.

In [13]:
function myjacobi_eig(A)
    n = size(A,1);    
    for i = 1:15
        # find s,t
        s,t = maxst(A);
        d = sqrt((A[s,s] - A[t,t])^2 + 4*A[s,t]^2);
        sin2t = 2*A[s,t]/d;
        cos2t = (A[s,s] - A[t,t]) / d;
        dt = sqrt(2*(1+cos2t));
        sint = abs(sin2t) / dt;
        cost = abs((1+ cos2t) / dt);
        cost = sign(A[s,t]) * cost;
        
        R = eye(n);
        R[s,s] = cost;
        R[t,t] = cost;
        R[s,t] = -sint;
        R[t,s] = sint;
        A = R'*A*R;
    end
    return A
end

myjacobi_eig (generic function with 1 method)

Testujemy kod obrotów jakobiego pamiętając, że macierz musi być symetryczna!

In [14]:
A = [12 6 -6 1
    6 16 2 100
    -6 2 16 1
    1 100 1 4];
maxst(A);
AJ = myjacobi_eig(A);

# tylko obcinamy prawie zera, żeby było lepiej widać
AJ[abs.(AJ) .< 1e-10] .= 0.0;
println("Macierz diagonalna:")
display(AJ)
e,v = eigen(A);
println("Wartośći własne z eigen: ", e)


4×4 Array{Float64,2}:
 110.475   0.0     0.0        0.0
   0.0    20.3274  0.0        0.0
   0.0     0.0     7.49633    0.0
   0.0     0.0     0.0      -90.2988

Macierz diagonalna:
Wartośći własne z eigen: [-90.2987517379167, 7.496326411004787, 20.32736609356303, 110.47505923334917]


Sprawdzamy jak w Julia działa przypisywanie wartości do elementów warunkowo.

In [15]:
A[ A .> 2 ] .= 100
A

4×4 Array{Int64,2}:
 100  100   -6    1
 100  100    2  100
  -6    2  100    1
   1  100    1  100